## Implementación de Kmeans - SENTINEL
### @utor: Romel Principe A #

## Script realiza : 

+ Carga datos Sentinel - Surface Reflectance
+ Corte del area 
+ Cálculo de índices físicos
+ Clasificación con Kmeans

### 1.- Autenticar e inicilizar

In [2]:
import ee
ee.Initialize() 

### 2.- Cargar funcion

In [3]:
# importando otras librerias 
import os
import geemap
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
#%matplotlib notebook

### 3.- Cargando datos Sentinel

In [4]:
# Poligono de interes
path = "./ShapeFile/POL_Cuenca_Chancay_AR_PEOT.shp"
#print(os.listdir(path))
poly = geemap.shp_to_ee(path)
# Extrayendo centroide
#print(poly)
coord = poly.getInfo()['features'][0]['geometry']['coordinates']
center_pol = ee.Geometry.Polygon(coord).centroid().getInfo()['coordinates']


In [6]:
#--------------------------------------------------------------
# Cargando el lienso
Map = geemap.Map(center= (center_pol[1],center_pol[0]), zoom=9.5)
Map.add_basemap("SATELLITE")
# Cargando poligono
Map.addLayer(poly, {}, 'Bofedal')
Map
#--------------------------------------------------------------

Map(center=[-6.692171087956604, -79.81498944300296], controls=(WidgetControl(options=['position'], widget=HBox…

In [53]:
# Parametro de fechas
# Level-2A
range_date = ['2020-10-01', '2020-10-15']
# Cargando la colección de imágenes filtrando con fecha y region de interes. 
collection_S = ee.ImageCollection("COPERNICUS/S2_SR")\
               .filterDate(range_date[0], range_date[1])\
               .filterBounds(poly).filter(ee.Filter.lt("CLOUDY_PIXEL_PERCENTAGE", 20))
# Contar numero de imagenes 
print('Sentinel: ', collection_S.size().getInfo()) 
#collection_S.getInfo()
#print(collection_S)
#print(collection_S.getInfo())

Sentinel:  1


In [7]:
# Cargando la imagen de interes.
# 2019/10/14
#img = ee.Image("COPERNICUS/S2_SR/20191014T153621_20191014T153622_T17MPN").divide(10000)
#2020/10/13
img = ee.Image("COPERNICUS/S2_SR/20201003T153619_20201003T153759_T17MPN").divide(10000)
#img.getInfo()
img.bandNames().getInfo()

['B1',
 'B2',
 'B3',
 'B4',
 'B5',
 'B6',
 'B7',
 'B8',
 'B8A',
 'B9',
 'B11',
 'B12',
 'AOT',
 'WVP',
 'SCL',
 'TCI_R',
 'TCI_G',
 'TCI_B',
 'MSK_CLDPRB',
 'MSK_SNWPRB',
 'QA10',
 'QA20',
 'QA60']

In [8]:
# Visualizando la imagen
#--------------------------------------------------------------
sentinel_vis = {
    'min': 0.0,
    'max': 1,
    'bands': ["B6", "B3", "B2"] }
#--------------------------------------------------------------
Map = geemap.Map(center=(center_pol[1],center_pol[0]), zoom=8.5)
Map.add_basemap("SATELLITE")
Map.addLayer(img, sentinel_vis , 'Sentinel RGB')
Map.addLayer(poly, {}, 'Cuenca')
Map
#--------------------------------------------------------------

Map(center=[-6.692171087956604, -79.81498944300296], controls=(WidgetControl(options=['position'], widget=HBox…

In [9]:
# Funcion de corte (image.clip())
#--------------------------------------------------------------
cut_img = img.clip(poly)
#--------------------------------------------------------------

In [11]:
#--------------------------------------------------------------
# Ploteando area de interes
Map_cut = geemap.Map(center=(center_pol[1],center_pol[0]), zoom=10.0)
Map_cut.add_basemap("SATELLITE")
Map_cut.addLayer(cut_img, sentinel_vis , 'Sentinel RGB')
#Map_cut.addLayer(poly, {}, 'area agricola')
Map_cut
#--------------------------------------------------------------

Map(center=[-6.692171087956604, -79.81498944300296], controls=(WidgetControl(options=['position'], widget=HBox…

### 4. Construyendo K-means Model

In [12]:
#--------------------------------------------------------------
# name bandas incluidas en el modelo
bands = ['B2', 'B3', 'B4', 'B5', 'B6', 'B7','B8','B8A','B11','B12'] 
# seleccionando las bandas que incluira en el modelo
sele_band = cut_img.select(bands)
# ensamblando las muestras para el modelo
muestra =  sele_band.sample(
    region = poly,
    numPixels = 5000,
    scale = 20 )
#
muestra
#--------------------------------------------------------------

In [13]:
# Instantiate the clusterer and train it.
n_clusters = 4
clusterer = ee.Clusterer.wekaKMeans(n_clusters).train(muestra)

### Classify the image

In [14]:
# Cluster the input using the trained clusterer.
result = sele_band.cluster(clusterer)

In [49]:
result.getInfo()

#  Display the clusters with random colors.
#Map.addLayer(result.randomVisualizer(), {}, 'clusters')
#Map

{'type': 'Image',
 'bands': [{'id': 'cluster',
   'data_type': {'type': 'PixelType',
    'precision': 'int',
    'min': -2147483648,
    'max': 2147483647},
   'dimensions': [10980, 10980],
   'crs': 'EPSG:32717',
   'crs_transform': [10, 0, 600000, 0, -10, 9300040]}]}

In [15]:
legend_keys = ['class1', 'class2', 'class3', 'class4']
legend_colors = ['466b9f', 'd1def8', 'd99282', 'eb0000']
# Reclassify the map
result_re = result.remap([0, 1, 2, 3 ], [1, 2, 3 ,4 ])
#
Map_clas = geemap.Map(center=(center_pol[1],center_pol[0]), zoom=9.5)
Map_clas.add_basemap("SATELLITE")
Map_clas.addLayer(cut_img, sentinel_vis , 'Sentinel RGB')
Map_clas.addLayer(poly, {}, 'area agricola')
Map_clas.addLayer(result_re, {min: 1, max: 5, 'palette': legend_colors}, 
                           'Labelled clusters')
Map_clas.add_legend(legend_title='Leyenda',legend_keys=legend_keys, 
               legend_colors=legend_colors, 
               position='bottomright')
Map_clas

Map(center=[-6.692171087956604, -79.81498944300296], controls=(WidgetControl(options=['position'], widget=HBox…

In [52]:
print('Change layer opacity:')
cluster_layer = Map_clas.layers[-1]
cluster_layer.interact(opacity=(0, 1, 0.1))

Change layer opacity:


Box(children=(FloatSlider(value=1.0, description='opacity', max=1.0),))

In [16]:
#--------------------------------------------------------------
# exportando solo una imagen
out_dir = os.path.join(os.path.expanduser('~'), 'Downloads')
filename = os.path.join(out_dir, 'CV_03_10_2020.tif') 
#
image = result
#roi = new_coord 
roi = poly.getInfo()['features'][0]['geometry']['coordinates']
geemap.ee_export_image(image, filename=filename, scale=30, region=roi, file_per_band=False)
#--------------------------------------------------------------

Generating URL ...
Please wait ...
Data downloaded to /Users/erickprincipeaguirre/Downloads/CV_03_10_2020.tif


In [59]:
# Otra manera - Descarga por link
roi = poly.getInfo()['features'][0]['geometry']['coordinates']
path = result_re.getDownloadURL({
    "scale": 20,
    "crs" : "EPSG:4326",
    "region": roi })
print(path)

https://earthengine.googleapis.com/v1alpha/projects/earthengine-legacy/thumbnails/36c988c370d92fbeaaf0ba50838b6489-a462b9369fd1d8322b926dbcba5f550a:getPixels
